# Load CSV data

This tutorial provides an example of how to load CSV data from a file into a `tf.data.Dataset`.

The data used in this tutorial are taken from the Titanic passenger list. The model will predict the likelihood a passenger survived based on characteristics like age, gender, ticket class, and whether the person was traveling alone.

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

## Load data

To start, let's look at the top of the CSV file to see how it is formatted.

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


You can [load this using pandas](pandas.ipynb), and pass the NumPy arrays to TensorFlow. If you need to scale up to a large set of files, or need a loader that integrates with [TensorFlow and tf.data](../../guide/data.ipynb) then use the `tf.data.experimental.make_csv_dataset` function:

The only column you need to identify explicitly is the one with the value that the model is intended to predict. 

In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

Now read the CSV data from the file and create a dataset. 

(For the full documentation, see `tf.data.experimental.make_csv_dataset`)


In [6]:
def get_dataset(filepath,**kwargs):
    dataset = tf.data.experimental.make_csv_dataset(filepath,batch_size=5,label_name=LABEL_COLUMN,na_value='?',
                                                    num_epochs=1,ignore_errors=True,**kwargs)
    return dataset

In [7]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


Each item in the dataset is a batch, represented as a tuple of (*many examples*, *many labels*). The data from the examples is organized in column-based tensors (rather than row-based tensors), each with as many elements as the batch size (5 in this case).

It might help to see this yourself.

In [8]:
def show_batch(dataset):
    for batch,label in dataset.take(1):
        for key, val in batch.items():
            print("{:20}:{}".format(key,val.numpy()))

In [9]:
show_batch(raw_test_data)

sex                 :[b'female' b'male' b'female' b'female' b'male']
age                 :[28. 52. 18.  8. 55.]
n_siblings_spouses  :[0 0 1 0 0]
parch               :[0 0 0 2 0]
fare                :[  8.137  13.5   227.525  26.25   30.5  ]
class               :[b'Third' b'Second' b'First' b'Second' b'First']
deck                :[b'unknown' b'unknown' b'C' b'unknown' b'C']
embark_town         :[b'Queenstown' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               :[b'y' b'y' b'n' b'n' b'y']


In [10]:
csv_columns = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=csv_columns)
show_batch(temp_dataset)

sex                 :[b'male' b'female' b'male' b'female' b'female']
age                 :[32.  22.  40.5 23.  51. ]
n_siblings_spouses  :[0 0 0 0 1]
parch               :[0 0 2 0 0]
fare                :[ 7.854 10.517 14.5   13.792 77.958]
class               :[b'Third' b'Third' b'Third' b'Second' b'First']
deck                :[b'unknown' b'unknown' b'unknown' b'D' b'D']
embark_town         :[b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               :[b'y' b'y' b'n' b'y' b'n']


This example is going to use all the available columns. If you need to omit some columns from the dataset, create a list of just the columns you plan to use, and pass it into the (optional) `select_columns` argument of the constructor.

In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 :[51. 24. 22. 28. 28.]
n_siblings_spouses  :[0 2 0 0 1]
class               :[b'First' b'Second' b'Third' b'Third' b'Third']
deck                :[b'E' b'unknown' b'unknown' b'unknown' b'unknown']
alone               :[b'y' b'n' b'y' b'n' b'n']


## Data preprocessing

A CSV file can contain a variety of data types. Typically you want to convert from those mixed types to a fixed length vector before feeding the data into your model.

TensorFlow has a built-in system for describing common input conversions: `tf.feature_column`, see [this tutorial](../keras/feature_columns) for details.


You can preprocess your data using any tool you like (like [nltk](https://www.nltk.org/) or [sklearn](https://scikit-learn.org/stable/)), and just pass the processed output to TensorFlow. 


The primary advantage of doing the preprocessing inside your model is that when you export the model it includes the preprocessing. This way you can pass the raw data directly to your model.

### Continuous data

If your data is already in an appropriate numeric format, you can pack the data into a vector before passing it off to the model:

In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 :[43. 18. 26. 28. 33.]
n_siblings_spouses  :[1. 0. 1. 0. 1.]
parch               :[1. 0. 1. 0. 2.]
fare                :[26.25   7.775 26.    13.    27.75 ]


In [14]:
example_batch, labels_batch=next(iter(temp_dataset))

In [15]:
example_batch

OrderedDict([('age',
              <tf.Tensor: id=439, shape=(5,), dtype=float32, numpy=array([28., 30., 24., 28., 24.], dtype=float32)>),
             ('n_siblings_spouses',
              <tf.Tensor: id=441, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 1.], dtype=float32)>),
             ('parch',
              <tf.Tensor: id=442, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 2.], dtype=float32)>),
             ('fare',
              <tf.Tensor: id=440, shape=(5,), dtype=float32, numpy=array([30.696,  8.663,  8.05 ,  6.95 , 65.   ], dtype=float32)>)])

In [16]:
labels_batch

<tf.Tensor: id=443, shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 1], dtype=int32)>

In [18]:
# next(iter(temp_dataset))

Here's a simple function that will pack together all the columns:

In [23]:
def pack(feature,label):
    return tf.stack(list(feature.values()),axis=-1),label

In [24]:
packed_dataset = temp_dataset.map(pack)

In [31]:
for feature,label in packed_dataset.take(1):
    print(feature.numpy())
    print()
    print(label.numpy())

[[ 34.      0.      0.      6.496]
 [ 28.      0.      0.     26.55 ]
 [ 24.      0.      1.    247.521]
 [ 44.      1.      0.     26.   ]
 [ 47.      0.      0.     52.   ]]

[0 1 0 0 0]


If you have mixed datatypes you may want to separate out these simple-numeric fields. The `tf.feature_column` api can handle them, but this incurs some overhead and should be avoided unless really necessary. Switch back to the mixed dataset:

In [32]:
show_batch(raw_train_data)

sex                 :[b'female' b'female' b'male' b'male' b'female']
age                 :[29.  5. 20. 28. 23.]
n_siblings_spouses  :[1 2 0 0 0]
parch               :[1 1 0 0 0]
fare                :[10.462 19.258  9.5    7.225 13.792]
class               :[b'Third' b'Third' b'Third' b'Third' b'Second']
deck                :[b'G' b'unknown' b'unknown' b'unknown' b'D']
embark_town         :[b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg' b'Cherbourg']
alone               :[b'n' b'n' b'y' b'y' b'y']


In [33]:
example_batch, labels_batch = next(iter(temp_dataset)) 

So define a more general preprocessor that selects a list of numeric features and packs them into a single column:

In [55]:
class PackNumericFeatures(object):
    
    def __init__(self, names):
        self.names = names



    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        return features, labels

In [56]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

[<tf.Tensor 'args_0:0' shape=(None,) dtype=float32>, <tf.Tensor 'args_6:0' shape=(None,) dtype=int32>, <tf.Tensor 'args_7:0' shape=(None,) dtype=int32>, <tf.Tensor 'args_5:0' shape=(None,) dtype=float32>]
[<tf.Tensor 'args_0:0' shape=(None,) dtype=float32>, <tf.Tensor 'args_6:0' shape=(None,) dtype=int32>, <tf.Tensor 'args_7:0' shape=(None,) dtype=int32>, <tf.Tensor 'args_5:0' shape=(None,) dtype=float32>]


In [57]:
show_batch(packed_train_data)

sex                 :[b'male' b'male' b'male' b'female' b'male']
class               :[b'Second' b'First' b'First' b'First' b'Third']
deck                :[b'unknown' b'A' b'C' b'B' b'unknown']
embark_town         :[b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               :[b'y' b'y' b'n' b'y' b'y']
numeric             :[[ 28.      0.      0.      0.   ]
 [ 80.      0.      0.     30.   ]
 [ 19.      3.      2.    263.   ]
 [ 29.      0.      0.    211.337]
 [ 20.      0.      0.      9.225]]


In [58]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#### Data Normalization

Continuous data should always be normalized.

In [61]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [65]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [66]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


Now create a numeric column. The `tf.feature_columns.numeric_column` API accepts a `normalizer_fn` argument, which will be run on each batch.

Bind the `MEAN` and `STD` to the normalizer fn using [`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial).

In [67]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f71fa57da60>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

When you train the model, include this feature column to select and center this block of numeric data:

In [68]:
example_batch['numeric']

<tf.Tensor: id=1158, shape=(5, 4), dtype=float32, numpy=
array([[ 27.   ,   0.   ,   0.   ,  76.729],
       [ 19.   ,   1.   ,   0.   ,  26.   ],
       [ 36.   ,   0.   ,   1.   , 512.329],
       [ 31.   ,   1.   ,   1.   ,  26.25 ],
       [ 28.   ,   0.   ,   0.   ,   7.225]], dtype=float32)>

In [69]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.21 , -0.474, -0.479,  0.776],
       [-0.85 ,  0.395, -0.479, -0.154],
       [ 0.509, -0.474,  0.782,  8.754],
       [ 0.109,  0.395,  0.782, -0.149],
       [-0.13 , -0.474, -0.479, -0.497]], dtype=float32)

The mean based normalization used here requires knowing the means of each column ahead of time.

### Categorical data

Some of the columns in the CSV data are categorical columns. That is, the content should be one of a limited set of options.

Use the `tf.feature_column` API to create a collection with a `tf.feature_column.indicator_column` for each categorical column.



In [70]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [71]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [72]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [73]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]


This will be become part of a data processing input later when you build the model.

### Combined preprocessing layer

Add the two feature column collections and pass them to a `tf.keras.layers.DenseFeatures` to create an input layer that will extract and preprocess both input types:

In [74]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [75]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     1.     0.     0.     0.     0.     0.     1.     0.
  0.     0.     0.     0.     0.     1.     0.     0.    -0.21  -0.474
 -0.479  0.776  1.     0.   ]


## Build the model

Build a `tf.keras.Sequential`, starting with the `preprocessing_layer`.

In [123]:
from tensorflow.keras import regularizers
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.001), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,kernel_regularizer=regularizers.l2(0.001), activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [124]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [129]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 4ms/step - loss: 0.4370 - accuracy: 0.8341
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4418 - accuracy: 0.8325: 0s - loss: 0.4448 - accuracy
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4267 - accuracy: 0.8469
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4238 - accuracy: 0.8389
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4542 - accuracy: 0.8389
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4326 - accuracy: 0.8453
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4190 - accuracy: 0.8517
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4317 - accuracy: 0.8341
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4243 - accuracy: 0.8405
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - l

Once the model is trained, you can check its accuracy on the `test_data` set.

In [130]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 0s 3ms/step - loss: 0.4758 - accuracy: 0.8258


Test Loss 0.4758159894988222, Test Accuracy 0.8257575631141663


Use `tf.keras.Model.predict` to infer labels on a batch or a dataset of batches.

In [131]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: 98.99%  | Actual outcome:  DIED
Predicted survival: 99.28%  | Actual outcome:  SURVIVED
Predicted survival: 10.19%  | Actual outcome:  DIED
Predicted survival: 10.14%  | Actual outcome:  DIED
Predicted survival: 22.30%  | Actual outcome:  SURVIVED
